# **Feature Engineering - installments_payments**

### **Importar Bibliotecas**

In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1701653078561_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Importar tabela 'installments_payments'.**

In [3]:
spark = SparkSession.builder.appName('Feature Engineering').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1701653078561_0001/container_1701653078561_0001_01_000001/pyspark.zip/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

In [4]:
file='installments_payments.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
S3_DATA_SOURCE_PATH = 'pod-academy-analise-de-credito-para-fintech/dados'
S3_DATA_OUTPUT_PATH = 'pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
output = 'abt_installments_payments'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
full_path_source = 's3://{bkt}/{file}'.format(bkt=S3_DATA_SOURCE_PATH,file=file)
print(full_path_source)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/dados/installments_payments.csv

In [8]:
installments_payments_00 = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_source)
installments_payments_00.registerTempTable("installments_payments_00")
installments_payments_00.cache()
qtd=installments_payments_00.count()
print('records ingestion data:', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingestion data: 13605401
/mnt1/yarn/usercache/livy/appcache/application_1701653078561_0001/container_1701653078561_0001_01_000001/pyspark.zip/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.

In [9]:
installments_payments_00.show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|1054186   |161674    |1.0                   |6                    |-1180.0        |-1187.0           |6948.36       |6948.36    |
|1330831   |151639    |0.0                   |34                   |-2156.0        |-2156.0           |1716.525      |1716.525   |
|2085231   |193053    |2.0                   |1                    |-63.0          |-63.0             |25425.0       |25425.0    |
|2452527   |199697    |1.0                   |3                    |-2418.0        |-2426.0           |24350.13      |24350.13   |
|2714724   |167756    |1.0                   |2                    |-1383.0        

In [10]:
installments_payments_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- SK_ID_PREV: string (nullable = true)
 |-- SK_ID_CURR: string (nullable = true)
 |-- NUM_INSTALMENT_VERSION: string (nullable = true)
 |-- NUM_INSTALMENT_NUMBER: string (nullable = true)
 |-- DAYS_INSTALMENT: string (nullable = true)
 |-- DAYS_ENTRY_PAYMENT: string (nullable = true)
 |-- AMT_INSTALMENT: string (nullable = true)
 |-- AMT_PAYMENT: string (nullable = true)

### **Comentário**

- Cada "SK_ID_PREV" se refere a um único "SK_ID_CURR".

- Um "SK_ID_CURR" pode corresponder a diferentes "SK_ID_PREV".

- Ou seja, podemos referenciar as variáveis criadas com a tabela "application", assim como, a tabela "previous_application".

In [11]:
# installments_payments_00 = spark.sql('''
# SELECT
#   *
# FROM
#   installments_payments_00
# WHERE
#   SK_ID_CURR = '183431'
# ''')

# installments_payments_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# # Contar valores distintos da variável SK_ID_CURR
# qtde_distinct_sk_id_curr = spark.sql('''
# SELECT
#   COUNT(DISTINCT SK_ID_CURR) AS qtde_distinct_sk_id_curr
# FROM
#   installments_payments_00
# ''')

# # Exibir a quantidade de valores distintos
# qtde_distinct_sk_id_curr.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Criação de Variáveis Auxiliares.**

Para auxiliar no entendimento dos dados da tabela em questão, foram criadas duas novas colunas com as seguintes regras:
* DELAYED_DAYS
 * WHEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT > 0 THEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT
 * WHEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT < 0 THEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT
 * ELSE 0
* STATUS
 * WHEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT > 0 THEN 'Atrasado'
 * WHEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT < 0 THEN 'Adiantado'
 * WHEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT = 0 THEN 'Ok'
 * ELSE 'Outro'

In [13]:
installments_payments_01 = spark.sql('''
SELECT
    *,
    CASE
        WHEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT > 0 THEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT
        WHEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT < 0 THEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT
        ELSE 0
    END AS DELAYED_DAYS,
    CASE
        WHEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT > 0 THEN 'Atrasado'
        WHEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT < 0 THEN 'Adiantado'
        WHEN DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT = 0 THEN 'Ok'
        ELSE 'Outro'
    END AS STATUS
FROM installments_payments_00;
''')

installments_payments_01.createOrReplaceTempView("installments_payments_01")
installments_payments_01.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13605401

# **Criação de variáveis preditivas**

In [14]:
# Definir funções de agregações desejadas
agg = ['SUM', 'MAX', 'MIN', 'AVG']

var_categoricas = ['STATUS', 'NUM_INSTALMENT_VERSION']
var_numericas = ['NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT', 'AMT_INSTALMENT', 'AMT_PAYMENT', 'DELAYED_DAYS']

# # Definir flags de meses
# meses_flag = ['ultimos_3_meses_flag', 'ultimos_6_meses_flag', 'ultimos_9_meses_flag', 'ultimos_12_meses_flag']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Criando variáveis explicativas de primeira camada**

- usar média, mínimo, soma, máximo
- considerar variáveis categóricas e variáveis numéricas

### **Gerar variáveis.**

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, round

df_original_spark = installments_payments_01

# Criar uma sessão do Spark
spark = SparkSession.builder.appName("AdaptacaoCodigo").getOrCreate()

# Inicializar o contador
contador_resultados = 0

# Criar lista para armazenar os resultados
resultados = []

for ag in agg:
    for var_cat in var_categoricas:
        for dom_cat in df_original_spark.select(var_cat).distinct().collect():
            dom_cat_value = dom_cat[var_cat]
            for var_numerica in var_numericas:
                resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_{var_numerica.lower()}_{var_cat.lower()}_{dom_cat_value.lower().replace(' ', '_').replace('.', '')},"

                # Adicionar o resultado à lista
                resultados.append(resultado)

                # for mes_flag in meses_flag:
                #     resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' AND {mes_flag} = 1 THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_U{mes_flag.split('_')[1]}M_{var_numerica.lower()[:5]}_{var_cat.lower()[:4]}_{dom_cat_value.lower()[:3].replace(' ', '_')},"

                #     # Adicionar o resultado à lista
                #     resultados.append(resultado)

                # Incrementar o contador
                contador_resultados += 1

# Printar o total de combinações
print(f"Total de combinações: {contador_resultados}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total de combina??es: 1656

In [16]:
# Inicializa a lista para armazenar os resultados
resultados_formatados = []

# Se há pelo menos um resultado na lista
if resultados:
    # Remove a vírgula do último item
    resultados[-1] = resultados[-1].rstrip(',')

    # Adiciona os resultados formatados à lista
    resultados_formatados.extend(resultados)

    # Une a lista em uma string usando uma quebra de linha como separador
    resultados_str = '\n'.join(resultados_formatados)
#     print(resultados_str)
else:
    resultados_str = "A lista de resultados está vazia."

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Colar variáveis criadas no código PySpark-SQL.**

In [17]:
installments_payments_02 = spark.sql(f'''
SELECT
  SK_ID_CURR as SK_ID_CURR_installments_payments,
  {resultados_str}
FROM
  installments_payments_01
GROUP BY
  SK_ID_CURR
''')

installments_payments_02.createOrReplaceTempView("installments_payments_02")
installments_payments_02.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

339587

In [18]:
# installments_payments_02.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
print(len(installments_payments_02.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1657

## **Salvando tabela no diretório do S3.**

In [20]:
# Save to parquet
path_output = os.path.join('s3://',S3_DATA_OUTPUT_PATH,output)
print(path_output)
installments_payments_02.coalesce(1).write.\
parquet(path_output,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application/abt_installments_payments

In [21]:
# abt_installments_payments=sqlContext.read.format("parquet").load('s3://',S3_DATA_OUTPUT_PATH,output)
# abt_installments_payments.registerTempTable("abt_installments_payments")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# abt_installments_payments.show(5,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…